# Effect Translator Agent

## 1. Load Song Metadata

In [ ]:
from shared.models.song_metadata import SongMetadata
song = SongMetadata('born_slippy', songs_folder='/home/darkangel/ai-light-show/songs')

## 2. Load Fixtures

In [ ]:

from backend.models.fixtures import FixturesListModel
from backend.services.dmx.dmx_canvas import DmxCanvas
from backend.config import FIXTURES_FILE
dmx_canvas = DmxCanvas()
fixtures = FixturesListModel(fixtures_config_file=FIXTURES_FILE, dmx_canvas=dmx_canvas)

## 3. Load Effect Translator Data

In [ ]:
fixtures_details = []
for fixture in fixtures:
    fd = fixture.to_dict()
    fd["action_list"] = []
    for action in fixture.actions:
        if action=="arm":
            continue
        fd["action_list"].append(action)
    fixtures_details.append(fd)


In [ ]:
actions_list = {}
for fixture in fixtures:
    for action_name, data in fixture.action_models.items():
        actions_list[action_name] = data

In [ ]:
beats = song.get_beats_array() 
beats = [beat for beat in beats if beat < 35]  # Filter out beats that are too short

In [ ]:
from pathlib import Path
templates_folder = Path('/home/darkangel/ai-light-show/backend/services/agents/prompts')
prompt_template = 'effect_translator.j2'

# Load the Jinja2 template
import jinja2
template_loader = jinja2.FileSystemLoader(searchpath=templates_folder)
template_env = jinja2.Environment(loader=template_loader)
template = template_env.get_template(prompt_template)

# render
instructions = template.render(song=song, fixtures=fixtures_details, actions_list=actions_list, beats=beats)

## 4. Call Ollama Server

In [ ]:
# Set up Ollama API parameters
OLLAMA_URL = "http://localhost:11434"
OLLAMA_MODEL = "gemma3n:e4b"

prompt = "a flash from left to right on every beat from start to 12 seconds"
full_prompt = f"{instructions}\n\nUser Request: {prompt}"

In [ ]:

# Call the Ollama server and show response
import requests
import json

def call_ollama(instructions, prompt, model=OLLAMA_MODEL, url=OLLAMA_URL):
    """
    Send a prompt to the Ollama server and return the response
    
    Args:
        instructions (str): The system instructions/context for the model
        prompt (str): The user prompt to send to the model
        model (str): The Ollama model to use
        url (str): The URL of the Ollama server
        
    Returns:
        str: The model's response text
    """
    api_endpoint = f"{url}/api/generate"
    
    # Combine instructions and prompt
    full_prompt = f"{instructions}\n\nUser Request: {prompt}"

    payload = {
        "model": model,
        "prompt": full_prompt,
        "stream": False
    }
    
    try:
        response = requests.post(api_endpoint, json=payload)
        response.raise_for_status()  # Raise exception for HTTP errors
        
        result = response.json()
        return result.get("response", "No response found")

    except requests.exceptions.RequestException as e:
        return f"Error calling Ollama API: {str(e)}"

# Use our rendered template as the prompt
print(f"Calling {OLLAMA_MODEL} with the rendered template...")
response = call_ollama(instructions, prompt)
print(f"Response received. Length: {len(response)} characters")

In [ ]:
print(response)